In [1]:
#Import libraries
import pandas as pd
import numpy as np

#For visualization
import matplotlib.pyplot as plt
import seaborn as sns
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

In [2]:
#Make a dataframe to read the data in DataSource1.csv
transactionsDf = pd.read_csv("DataSource1.csv")
transactionsDf.head(5)

,Retailer country,Retailer type,Order method type,Product line,Product type,Product,Year,Quarter,Revenue,Quantity,Gross margin,Retailer ID
0,Australia,Department Store,Telephone,Camping Equipment,Cooking Gear,TrailChef Water Bag,2012,Q1 2012,10824.30,1710,0.537125,2
1,Australia,Department Store,Telephone,Camping Equipment,Cooking Gear,TrailChef Canteen,2012,Q1 2012,7007.23,799,0.496009,9
2,Australia,Department Store,Telephone,Camping Equipment,Cooking Gear,TrailChef Cup,2012,Q1 2012,2785.84,776,0.763231,2
3,Australia,Department Store,Telephone,Camping Equipment,Cooking Gear,TrailChef Kettle,2012,Q1 2012,38896.65,4905,0.496847,4
4,Australia,Department Store,Telephone,Camping Equipment,Tents,Star Gazer 6,2012,Q1 2012,18176.67,23,0.379974,3


In [3]:
#Make a dataframe to read the data in DataSource2.csv
salesDf = pd.read_csv("DataSource2.csv")
salesDf.head(5)

,Gender,Name,Hours Worked,SalesTraining,Years Experience,Staff Id,Shop Id
0,M,Pricha,39.0,N,3.0,1.0,17.0
1,M,Li,46.0,N,3.0,2.0,106.0
2,F,Erity,42.0,Y,4.0,3.0,113.0
3,M,Bayana,38.0,Y,5.0,4.0,235.0
4,M,Yongnian,33.0,N,4.0,5.0,278.0


In [4]:
#Check whether there are null values in the data frames created.
print(transactionsDf.isnull().sum())
print('\n')
print(salesDf.isnull().sum())

Retailer country       0
Retailer type          0
Order method type      0
Product line           0
Product type           0
Product                0
Year                   0
Quarter                0
Revenue                0
Quantity               0
Gross margin         581
Retailer ID            0
dtype: int64


Gender               4
Name                10
Hours Worked         3
SalesTraining        3
Years Experience     3
Staff Id             3
Shop Id              3
dtype: int64


In [5]:
#Question a-(i)
#Which are the top three performing retailer countries in terms of revenue and gross margin? 
#What are the amounts achived during the period?

#Although there are null values in "Gross margin" column, we do not need to remove 
#the rows containing thoses values since the question asks us to find the top three performing retailer 
#countries in terms of revenue and gross margin which are unilateral.

#first let's find the top three performing retailer countries in terms of "Revenue".
revDf = transactionsDf.groupby(['Retailer country'])['Revenue'].sum().reset_index()
revDf.sort_values(by=['Revenue'], inplace=True,ascending=False)
print(revDf.iloc[:3])

#put a gap
print('\n')

#Next let's find the top three performing retailer countries in terms of "Gross margin".
grossDf = transactionsDf.groupby(['Retailer country'])['Gross margin'].sum().reset_index()
grossDf.sort_values(by=['Gross margin'], inplace=True,ascending=False)
print(grossDf.iloc[:3])

   Retailer country       Revenue
20    United States  6.508110e+08
11            Japan  2.816655e+08
5             China  2.488232e+08


   Retailer country  Gross margin
20    United States   3348.866446
4            Canada   2662.767825
8            France   2590.572647


In [6]:
#Question a-(ii)
#What are the top three profitable products sold? What are the
#top three countries that sold each of these products?

#We have to find the top three products with most revenue.
productDf = transactionsDf.copy()
productDf.sort_values(by=['Revenue'], inplace=True,ascending=False)
print(productDf.iloc[0:3,[0,5,8]]) #Add 3 and 4 in the second list inside iloc if you want "Product line" and "Product type" too.

      Retailer country                       Product     Revenue
81777    United States  Hailstorm Titanium Woods Set  1635687.96
81704    United States                  Star Gazer 2  1486717.10
82315    United States                     Star Lite  1415141.91


In [7]:
#Question a-(iii)
#Which is the most productive order method(i.e. the average revenue is higher?)
orderMethodDf = transactionsDf.groupby(['Order method type'])['Revenue','Quantity'].sum().reset_index()
orderMethodDf['avgRevenue']= np.where(orderMethodDf['Quantity'] < 1, orderMethodDf['Quantity'], orderMethodDf['Revenue']/orderMethodDf['Quantity'])
orderMethodDf.sort_values(by=['avgRevenue'], inplace=True,ascending=False)
print(orderMethodDf.iloc[0:3,[0,1,3]])

  Order method type       Revenue  avgRevenue
6               Web  3.238464e+09   55.446061
1               Fax  4.143407e+07   52.366789
0            E-mail  8.444025e+07   50.982360


<ipython-input-7-635076cdcf3c>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  orderMethodDf = transactionsDf.groupby(['Order method type'])['Revenue','Quantity'].sum().reset_index()


In [85]:
#Question b-(i)
#By further enchancing your program in Question Q4(a), formulate new funcitons that answer the following questions.
#Gross margin contribution is defined as margin(%) multiplied by revenue.
#Using this formula, find out what is the gross margin contribution amount(i.e. in $)
#per employee for the top performing Retailer(by gross margin amount) in each country?

salesDf=salesDf.dropna(how='all')
empDf=salesDf.groupby(['Shop Id'])["Staff Id"].count().reset_index()
empDf["empCount"]=empDf["Staff Id"]
empDf["Retailer ID"]=empDf["Shop Id"]
empDf["Retailer ID"]=empDf["Retailer ID"].astype(int)
empDf=empDf.drop(columns=["Staff Id", "Shop Id"])

retailerDf = transactionsDf.groupby(["Retailer country","Retailer ID"])["Revenue", "Gross margin"].sum().reset_index()

#retailerDf["Retailer country"] = retailerDf["Retailer country"].apply(lambda col: col[1].str([0]))
concatDf= pd.merge(retailerDf,empDf,on='Retailer ID')

#calculate gross margin revenue
concatDf['gmRev']=np.where(concatDf['Revenue'] < 1, concatDf['Revenue'], concatDf['Revenue']*concatDf['Gross margin'])
#calculatr gross margin revenue per employee
concatDf['gmRevEmp']=np.where(concatDf['empCount'] < 1, concatDf['empCount'], concatDf['gmRev']/concatDf['empCount'])
concatDf.sort_values(by=['gmRevEmp'], inplace=True,ascending=False)

#Since we want the top retail id from each country, we are going to remove the duplicates based on "Retail country"
retailTopDf = concatDf.copy()
retailTopDf=retailTopDf.drop_duplicates(subset=["Retailer country"],keep="first",inplace=False)
#Since we only want "Retailer country" and "Retailer ID" we will drop the other columns
retailTopDf=retailTopDf.drop(columns=["Revenue","Gross margin","empCount","gmRev","gmRevEmp"])
print(retailTopDf)

    Retailer country  Retailer ID
149            Japan          157
103           France          109
279    United States          292
13           Austria           15
2          Australia            3
171            Korea          179
177           Mexico          185
206            Spain          214
197        Singapore          205
38            Canada           40
19           Belgium           21
140            Italy          146
119          Germany          125
101          Finland          107
188      Netherlands          196
246   United Kingdom          258
32            Brazil           34
64             China           67
227      Switzerland          238
217           Sweden          227
91           Denmark           96


<ipython-input-85-e14014dde454>:14: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  retailerDf = transactionsDf.groupby(["Retailer country","Retailer ID"])["Revenue", "Gross margin"].sum().reset_index()


In [82]:
fx=concatDf.groupby(["Retailer country"])
fx.head()

,Retailer country,Retailer ID,Revenue,Gross margin,empCount,gmRev,gmRevEmp
149,Japan,157,19532053.97,161.646421,1,3.157287e+09,3.157287e+09
103,France,109,14960165.94,188.002728,1,2.812552e+09,2.812552e+09
151,Japan,159,17935594.78,147.553457,1,2.646459e+09,2.646459e+09
279,United States,292,21523886.15,114.594268,1,2.466514e+09,2.466514e+09
153,Japan,161,15416024.70,153.794167,1,2.370895e+09,2.370895e+09
...,...,...,...,...,...,...,...
215,Sweden,225,5957851.20,115.469513,3,6.879502e+08,2.293167e+08
92,Denmark,97,4685585.45,108.148923,3,5.067410e+08,1.689137e+08
88,Denmark,93,4487749.82,106.812704,3,4.793487e+08,1.597829e+08
93,Denmark,98,5431774.55,116.546819,4,6.330560e+08,1.582640e+08


In [95]:
#Question b-(ii)
#What is the average working experience of staff in each country?

expDf=salesDf.groupby(['Shop Id'])["Years Experience"].sum().reset_index()
expDf["Shop Id"]=expDf["Shop Id"].astype(int)
transDf=transactionsDf.copy()
transDf['Shop Id']=transDf["Retailer ID"]
transDf=transDf.drop(columns="Retailer ID")
df = pd.merge(expDf,transDf,on="Shop Id")
#Since all the shops having the "same id" has gained the "years experience" same
df=df.drop_duplicates(subset=["Shop Id"],keep="first",inplace=False)
avgExpDf = df.groupby(["Retailer country"])["Years Experience"].mean().reset_index()
avgExpDf["average experience"]=avdExpDf["Years Experience"]
avgExpDf=avgExpDf.drop(columns=["Years Experience"])
print(avgExpDf)

   Retailer country  average experience
0         Australia           10.888889
1           Austria           11.500000
2           Belgium           10.571429
3            Brazil            9.727273
4            Canada           12.047619
5             China            8.888889
6           Denmark           12.444444
7           Finland           10.500000
8            France            9.928571
9           Germany           10.454545
10            Italy           12.333333
11            Japan           11.666667
12            Korea            9.833333
13           Mexico            9.750000
14      Netherlands           11.090909
15        Singapore           11.888889
16            Spain           12.833333
17           Sweden           10.300000
18      Switzerland           13.076923
19   United Kingdom            9.416667
20    United States           11.892857


In [97]:
#Question b-(iii)
#What is the total working hours of staff in each retail store?

workingHrsDf = salesDf.groupby(['Shop Id'])["Hours Worked"].sum().reset_index()
workingHrsDf['Shop Id']=workingHrsDf['Shop Id'].astype(int)
print(workingHrsDf)

     Shop Id  Hours Worked
0          1          38.0
1          2         107.0
2          3          37.0
3          4         248.0
4          5         176.0
..       ...           ...
282      295         154.0
283      296         146.0
284      298          58.0
285      299         105.0
286      300         139.0

[287 rows x 2 columns]


In [ ]:
#Question - c
#The sales director is considering investing in more sales training or hiring more experience
#staff to improve sales. What would you suggest and why?

#Looking at the above results we can deduce that the companies with more machinery rather than employees
#have been earning a considerable revenue than the companies which are prioritizing the employees over machinery.
#And if the company we are considering is not a machine based company we can say that thses datasets are biased because this dataset even have companies with one employee. 